In [1]:
import tensorflow as tf
from tools import get_file, get_data_file
from PIL import Image

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(get_file('bin/inference_graph/frozen_inference_graph.pb'), 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

In [4]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

In [5]:

IMAGE_SIZE = (10, 10)

In [6]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[1], image.shape[2])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: image})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.int64)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

In [7]:
import sys
print(sys.path)
sys.path.append('/Users/antonioferegrino/Documents/GitHub/salsa-valentina/src/external')

['/Users/antonioferegrino/Documents/GitHub/salsa-valentina/src', '/Users/antonioferegrino/.local/share/virtualenvs/salsa-valentina-jMmJ4Jum/lib/python36.zip', '/Users/antonioferegrino/.local/share/virtualenvs/salsa-valentina-jMmJ4Jum/lib/python3.6', '/Users/antonioferegrino/.local/share/virtualenvs/salsa-valentina-jMmJ4Jum/lib/python3.6/lib-dynload', '/Users/antonioferegrino/anaconda3/lib/python3.6', '', '/Users/antonioferegrino/.local/share/virtualenvs/salsa-valentina-jMmJ4Jum/lib/python3.6/site-packages', '/Users/antonioferegrino/.local/share/virtualenvs/salsa-valentina-jMmJ4Jum/lib/python3.6/site-packages/IPython/extensions', '/Users/antonioferegrino/.ipython']


In [8]:

print(sys.path)

['/Users/antonioferegrino/Documents/GitHub/salsa-valentina/src', '/Users/antonioferegrino/.local/share/virtualenvs/salsa-valentina-jMmJ4Jum/lib/python36.zip', '/Users/antonioferegrino/.local/share/virtualenvs/salsa-valentina-jMmJ4Jum/lib/python3.6', '/Users/antonioferegrino/.local/share/virtualenvs/salsa-valentina-jMmJ4Jum/lib/python3.6/lib-dynload', '/Users/antonioferegrino/anaconda3/lib/python3.6', '', '/Users/antonioferegrino/.local/share/virtualenvs/salsa-valentina-jMmJ4Jum/lib/python3.6/site-packages', '/Users/antonioferegrino/.local/share/virtualenvs/salsa-valentina-jMmJ4Jum/lib/python3.6/site-packages/IPython/extensions', '/Users/antonioferegrino/.ipython', '/Users/antonioferegrino/Documents/GitHub/salsa-valentina/src/external']


In [9]:

from external.object_detection.utils import visualization_utils as vis_util

In [12]:
image = Image.open(get_data_file('external', 'test1.jpg'))
# the array based representation of the image will be used later in order to prepare the
# result image with boxes and labels on it.
image_np = load_image_into_numpy_array(image)
# Expand dimensions since the model expects images to have shape: [1, None, None, 3]
image_np_expanded = np.expand_dims(image_np, axis=0)
# Actual detection.
output_dict = run_inference_for_single_image(image_np_expanded, detection_graph)
# Visualization of the results of a detection.
vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      {1:{'name':'Valentina'}},
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=8)
plt.figure(figsize=IMAGE_SIZE)
plt.imshow(image_np)
plt.savefig(f'test1.jpg')

In [11]:
plt.show()

/Users/antonioferegrino/.local/share/virtualenvs/salsa-valentina-jMmJ4Jum/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  """Entry point for launching an IPython kernel.
